# Data preparation

In this notebook we regroup all the results about the illustrations into a single dataframe.
We then clean the data to homogenize it to be able to use it easily.

## Merging infos

In [1]:
import pandas as pd
from deep_translator import GoogleTranslator
import math
import regex as re

We merge information from painting reproductions from Smartify (`illu_infos_clean`), Artnet (`artnet_data`), Wikimedia commons (`commons_data`) and the Orsay museum (`orsay_data`).

In [231]:
# Smartify data
known_paintings_df = pd.read_csv("./data/illu_infos_clean.csv")
known_paintings_df = known_paintings_df.set_index('id')

In [232]:
# Adding Artnet data
artnet = pd.read_csv("./data/artnet_data.csv")
artnet.columns = map(str.lower, artnet.columns)
artnet = artnet.set_index('id')
all_illus = known_paintings_df.append(artnet)

In [233]:
# Adding Wikimedia Commons data
commons = pd.read_csv("./data/commons_data.csv")
commons.columns = map(str.lower, commons.columns)
commons = commons.set_index('id')
all_illus = all_illus.append(commons)

In [234]:
# Adding Orsay data
orsay = pd.read_csv("./data/orsay_data.csv")
orsay.columns = map(str.lower, orsay.columns)
orsay = orsay.set_index('id')
all_illus = all_illus.append(orsay)

Because all the data that we are stacking is known to be painting reproductions (by construction), we add a column "Category" in which they all belong to "Reroduction".

In [235]:
all_illus['category'] = all_illus.apply(lambda row: "Reproduction", axis=1)

And finally we can add our knowledge of the illustrations' categories.

In [237]:
cats = pd.read_csv("./data/final_pred_cat.csv")
cats = cats.set_index('id')
cats = cats.rename(columns={'pred_cat': 'category'})

In [238]:
df = all_illus.append(cats)

We drop illustrations which are twice in de dataframe. By default, the first occurence is kept.

In [239]:
df = df[~df.index.duplicated()]

In [240]:
# Cleaning data illustrations names
df.index = df.index.str.replace(" \(1\)", "").str.replace("\(1\)", "")

C:\Users\emichelet\AppData\Local\Temp\ipykernel_6640\3837249030.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  df.index = df.index.str.replace(" \(1\)", "").str.replace("\(1\)", "")


We also enrich the metadata of the illustrations by adding the ID of the journal it comes from.

In [250]:
master = pd.read_csv('data/DFKV_Master.csv')
master

,ID,Volume_ID,_journal-id,liens iiif,liens de citation (page),liens de citation (volume),bibliographie
0,15573,8640,1411.0,NaN,NaN,https://gallica.bnf.fr/ark:/12148/bpt6k7522165...,supplément
1,14385,8640,1518.0,NaN,x,NaN,NaN
2,14389,8641,1568.0,NaN,NaN,https://gallica.bnf.fr/ark:/12148/bpt6k360915?...,NaN
3,14390,8642,1568.0,NaN,NaN,https://gallica.bnf.fr/ark:/12148/bpt6k36087x?...,NaN
4,14394,8643,1568.0,https://gallica.bnf.fr/iiif/ark:/12148/bpt6k36...,https://gallica.bnf.fr/ark:/12148/bpt6k36008s/...,NaN,NaN
...,...,...,...,...,...,...,...
6323,14457,8636,1463.0,https://gallica.bnf.fr/iiif/ark:/12148/bpt6k57...,https://gallica.bnf.fr/ark:/12148/bpt6k5780528...,NaN,S. 121-136
6324,14458,8637,1463.0,https://gallica.bnf.fr/iiif/ark:/12148/bpt6k57...,https://gallica.bnf.fr/ark:/12148/bpt6k5780528...,NaN,S. 287-304 (dernier article)
6325,14924,8638,1491.0,https://gallica.bnf.fr/iiif/ark:/12148/bpt6k30...,https://gallica.bnf.fr/ark:/12148/bpt6k3047055...,NaN,"4e année, n° 60, 1.7.1919, p. 473-475"
6326,14922,8639,1491.0,https://gallica.bnf.fr/iiif/ark:/12148/bpt6k30...,https://gallica.bnf.fr/ark:/12148/bpt6k3047050...,NaN,"4e année, n° 55, 1.2.1919, p. 433-436"


In [287]:
df['journal_id'] = df.apply(lambda row: int(list(master[master['ID'] == int(row.name.split('_')[1])]["_journal-id"])[0]), axis=1)

And finally, we save our results.

In [292]:
df.to_csv('data/merged_data.csv', index=True)

## Cleaning

Now, let's go through all the columns and clean them, so that their content is easy to use.

In [397]:
df = pd.read_csv('data/merged_data.csv').set_index('id')
df.sample(5)

,title,author,link,technique,dimensions,date,category,journal_id
id,,,,,,,,
ILLU_11516_325_0,Still Life with Apples,Paul Cézanne,NaN,Oil on canvas,46 x 55cm,1894,Reproduction,1438
ILLU_15313_189_0,NaN,NaN,NaN,NaN,NaN,NaN,Ornament,1476
ILLU_16302_332_2,NaN,NaN,NaN,NaN,NaN,NaN,Reproduction,1302
ILLU_14960_64_2,NaN,NaN,NaN,NaN,NaN,NaN,Reproduction,1473
ILLU_15305_21_0,NaN,NaN,NaN,NaN,NaN,NaN,Reproduction,1476


### Technique

For the 'technique' column, we will homogenize them by putting everything in lowercase, and translating everything back to english.

In [398]:
# translate
df['technique'] = df.apply(lambda row: GoogleTranslator(source='auto', target='en').translate(row['technique'])
                           if row['technique'] == row['technique']
                           else row['technique'], axis=1)

In [399]:
# lowercase everything
df['technique'] = df['technique'].str.lower()

In [400]:
df.groupby('technique').count().sort_values('category', ascending=False).head()

,title,author,link,dimensions,date,category,journal_id
technique,,,,,,,
oil on canvas,1034,1066,610,933,1033,1066,1066
oil on panel,39,39,27,31,36,39,39
oil,10,18,1,11,16,18,18
ink on paper,16,16,0,7,16,16,16
pastel,14,14,0,6,12,14,14


### Dimensions

The dimensions are more challenging, as the format used is very different from one row to another. We will : remove everything that's between parentheses, because usually it contains the conversion in inches. We also remove white spaces, unnecessary words ("Height:"), and put everything in centimeters and the same format : HxWcm

In [419]:
def clean_height_width(dimensions):
    if dimensions == dimensions: #Will be false if NaN
        if ('height' in dimensions and 'width' in dimensions) or ('H.' in dimensions and 'L.' in dimensions):
            hw = re.findall(r"[-+]?(?:\d*\.\d+|\d+)", dimensions)
            return hw[0] + 'x' + dimensions[dimensions.find(hw[1]) : dimensions.find(hw[1]) + len(hw[1]) + 2]
        elif 'height' in dimensions:
            h = re.findall(r"[-+]?(?:\d*\.\d+|\d+)", dimensions)[0]
            idx = dimensions.find(h)
            return dimensions[idx : idx + len(h) + 2]
        else:
            return dimensions
    else:
        return dimensions

In [488]:
def convert_in_cm(dimensions):
    if dimensions == dimensions:
        metric = re.findall("[a-zA-Z]+",  dimensions)[-1]
        values = re.findall(r"[-+]?(?:\d*\.\d+|\d+)",  dimensions)
        coeff = 2.54 if metric == "in" else 10 if metric == "m" else .1 if metric == "mm" else 1
        return "x".join([format(float(v) * coeff, '.2f') for v in values]) + "cm"
    else:
        return dimensions

In [490]:
# remove what's between parenthesis
df['dimensions'] = df['dimensions'].str.replace(r"\(.*\)","")
# remove white spaces
df['dimensions'] = df['dimensions'].str.replace(" ", "")
# deal with commas instead of points
df['dimensions'] = df['dimensions'].str.replace(",", ".")
# deal with :  \nheight: 47 cm (18.5 in) ; width: 38.4 cm 
df['dimensions'] = df.apply(lambda row: clean_height_width(row["dimensions"]), axis=1)
# conversion in cm
df['dimensions'] = df.apply(lambda row: convert_in_cm(row["dimensions"]), axis=1)
df.sample(10)

C:\Users\emichelet\AppData\Local\Temp\ipykernel_6640\1544484082.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  df['dimensions'] = df['dimensions'].str.replace(r"\(.*\)","")


,title,author,link,technique,dimensions,date,category,journal_id
id,,,,,,,,
ILLU_10467_498_0,Still Life with Herrings,Jean-Siméon Chardin,https://clevelandart.org/art/1974.1,oil on canvas,41.00x33.60cm,c. 1735,Reproduction,1438
ILLU_12372_379_1,NaN,NaN,NaN,NaN,NaN,NaN,Reproduction,1438
ILLU_12659_45_0,NaN,NaN,NaN,NaN,NaN,NaN,Reproduction,1336
ILLU_15409_360_1,NaN,NaN,NaN,NaN,NaN,NaN,Sculpture,1600
ILLU_12003_24256_2,NaN,NaN,NaN,NaN,NaN,NaN,Architecture_decoration,1384
ILLU_11309_37_0,Le pont de Mantes (environs de Paris),Camille Corot,https://donate.louvre.fr/particuliers/~mon-don,oil on canvas,3.80x5.50cm,Vers 1868 - 1870,Reproduction,1438
ILLU_10556_280_0,NaN,NaN,NaN,NaN,NaN,NaN,Reproduction,1368
ILLU_10535_169_0,NaN,NaN,NaN,NaN,NaN,NaN,Reproduction,1368
ILLU_10875_308_0,NaN,NaN,NaN,NaN,NaN,NaN,Sculpture,1438


### Date

Finally we create an additional column that will be used to sort chronologically the illustrations, thus we have to simplify to only keep one year. Example : "c. 1978" -> "1978"; "Between 1883 and 1990" -> "1883", ...

In [537]:
def simplify_date(date):
    if date == date:
        dates = re.findall('[0-9]+', date)
        if len(dates)>0:
            return dates[0]
        else:
            return float('NaN')
    else:
        return date

In [538]:
df['date_s'] = df.apply(lambda row: simplify_date(row["date"]), axis=1)

In [548]:
df.sample(5)

,title,author,link,technique,dimensions,date,category,journal_id,date_s
id,,,,,,,,,
ILLU_14377_346_2,Seated and Standing Woman,Georges Seurat,NaN,oil on wood,NaN,1884,Reproduction,1302,1884
ILLU_14279_124_0,After the Luncheon,Pierre-Auguste Renoir,https://commons.wikimedia.org/wiki/File:August...,oil on canvas,81.30x100.50cm,1879,Reproduction,1408,1879
ILLU_14957_195_5,NaN,NaN,NaN,NaN,NaN,NaN,Reproduction,1473,NaN
ILLU_15361_89_0,NaN,NaN,NaN,NaN,NaN,NaN,Reproduction,1279,NaN
ILLU_14405_40_1,NaN,NaN,NaN,NaN,NaN,NaN,Sculpture,1302,NaN


And we save the clean and complete dataframe.

In [549]:
df.to_csv("data/full_data_clean.csv")

## Prepare data for Vikov Viewer

In [17]:
df = pd.read_csv("data/full_data_clean.csv")
df = df.dropna(subset=['date_s'])
df = df.rename(columns={"date_s": "year"})
df['keywords'] = df['category']

In [23]:
df = df.rename(columns={"title":"_descrpiton", "technique": "_material", "dimensions": "_dimensions", "author":"_artist"})

In [27]:
df = df.drop(columns=['title', 'link', 'date', 'category', 'year', 'journal_id'])

In [28]:
df.to_csv("data/simple_dfkv_vikus.csv", index=False)

In [29]:
df

,id,_artist,_material,_dimensions,keywords,_description
0,ILLU_10332_143_0,Auguste Renoir,oil on canvas,NaN,Reproduction,Girl and Cat
1,ILLU_10346_257_0,Eugène Delacroix,oil on canvas,62.00x50.00cm,Reproduction,The death of Laras
2,ILLU_10359_147_0,Georges Seurat,oil on canvas,201.00x300.00cm,Reproduction,Une Baignade à Asnières
3,ILLU_10362_309_0,André Derain,oil on canvas,60.00x73.00cm,Reproduction,Landscape
4,ILLU_10365_17_0,Henri Matisse,oil on canvas,NaN,Reproduction,Figure at the Rideau Releve
...,...,...,...,...,...,...
1739,ILLU_12014_482_0,Edgar Degas,oil on canvas,0.40x0.50cm,Reproduction,Mademoiselle Dihau au piano
1740,ILLU_13055_13_0,Eugène Carrière,oil on canvas,0.12x0.50cm,Reproduction,La Famille du peintre
1741,ILLU_13061_275_0,Ferdinand Hodler,oil on canvas,0.13x0.50cm,Reproduction,Le Bûcheron
1742,ILLU_13291_21_0,Hans Thoma,oil on canvas,0.88x0.40cm,Reproduction,Siesta
